# Generate Karate API Tests with Llama 3.1

This notebook generates Karate API tests from an OpenAPI JSON specification using Llama 3.1 in Google Colab. Tests are saved as `.feature` files in `karate_tests`.

## Prerequisites
- Upload `openapi.json` to Colab.
- Ensure internet access for Ollama.
- Libraries: `json`, `os`, `argparse`, `ollama`.


In [ ]:
# Install and set up Ollama
!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

In [2]:
# Start Ollama server
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)



In [ ]:
# Pull Llama 3.1:8b
!ollama pull llama3.1:8b
#!ollama pull llama3.1:7b

In [ ]:
!pip install --upgrade langchain langchain_community

In [5]:
# Initialize Ollama client
from langchain.llms import Ollama
llm = Ollama(model="llama3.1:8b")

<ipython-input-5-5a2cbe471249>:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1:8b")


In [9]:
# Load OpenAPI JSON specification
import json
import os

def load_openapi_spec(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    with open(file_path, 'r') as file:
        return json.load(file)

# Load spec
try:
    spec = load_openapi_spec('openapi.json')
    print("OpenAPI spec loaded successfully")
except Exception as e:
    print(f"Error loading spec: {e}")

OpenAPI spec loaded successfully


In [10]:
# Extract endpoints from OpenAPI spec
from copy import deepcopy

def extract_referenced_schemas(schema, components_schemas, collected_schemas=None):
    if collected_schemas is None:
        collected_schemas = set()
    if not schema or not isinstance(schema, dict):
        return collected_schemas
    if '$ref' in schema:
        ref = schema['$ref']
        if ref.startswith('#/components/schemas/'):
            schema_name = ref.split('/')[-1]
            if schema_name not in collected_schemas:
                collected_schemas.add(schema_name)
                if schema_name in components_schemas:
                    schema_def = components_schemas[schema_name]
                    extract_referenced_schemas(schema_def, components_schemas, collected_schemas)
    else:
        if 'properties' in schema:
            for prop_value in schema['properties'].values():
                extract_referenced_schemas(prop_value, components_schemas, collected_schemas)
        if 'items' in schema:
            extract_referenced_schemas(schema['items'], components_schemas, collected_schemas)
        if 'additionalProperties' in schema and isinstance(schema['additionalProperties'], dict):
            extract_referenced_schemas(schema['additionalProperties'], components_schemas, collected_schemas)
        for combine_key in ['allOf', 'anyOf', 'oneOf']:
            if combine_key in schema:
                for sub_schema in schema[combine_key]:
                    extract_referenced_schemas(sub_schema, components_schemas, collected_schemas)
        for key, value in schema.items():
            if isinstance(value, dict):
                extract_referenced_schemas(value, components_schemas, collected_schemas)
            elif isinstance(value, list):
                for item in value:
                    extract_referenced_schemas(item, components_schemas, collected_schemas)
    return collected_schemas

def create_endpoint_spec(original_spec, path, method, endpoint_schemas):
    endpoint_spec = {
        'openapi': original_spec['openapi'],
        'info': original_spec['info'],
        'servers': original_spec['servers'],
        'paths': {path: {method: deepcopy(original_spec['paths'][path][method])}},
        'components': {'schemas': {}}
    }
    for schema_name in endpoint_schemas:
        if schema_name in original_spec['components']['schemas']:
            endpoint_spec['components']['schemas'][schema_name] = deepcopy(
                original_spec['components']['schemas'][schema_name]
            )
    return endpoint_spec

# Extract endpoints
endpoint_specs = []
components_schemas = spec.get('components', {}).get('schemas', {})
for path in spec['paths']:
    for method in spec['paths'][path]:
        endpoint_schemas = set()
        request_body = spec['paths'][path][method].get('requestBody', {})
        content = request_body.get('content', {}).get('application/json', {})
        if content.get('schema'):
            endpoint_schemas.update(extract_referenced_schemas(content['schema'], components_schemas))
        for response_code, response in spec['paths'][path][method].get('responses', {}).items():
            content = response.get('content', {}).get('application/json', {})
            if content.get('schema'):
                endpoint_schemas.update(extract_referenced_schemas(content['schema'], components_schemas))
        endpoint_spec = create_endpoint_spec(spec, path, method, endpoint_schemas)
        endpoint_specs.append({'spec': endpoint_spec, 'path': path, 'method': method})
print(f"Extracted {len(endpoint_specs)} endpoints")

Extracted 3 endpoints


In [ ]:
endpoint_specs

In [12]:
# Generate Karate tests with Llama 3.1
def clean_json_string(json_str):
    return ''.join(c for c in json_str if c.isprintable() or c in '\n\t')

def generate_karate_test(endpoint_spec, path, method, output_dir):
    try:
        endpoint_spec_json = json.dumps(endpoint_spec, indent=2, ensure_ascii=False)
        endpoint_spec_json = clean_json_string(endpoint_spec_json)
        json.loads(endpoint_spec_json)
    except json.JSONDecodeError as e:
        print(f"JSON error for {method.upper()} {path}: {e}")
        with open(f"{output_dir}/debug_{path.replace('/', '_')}_{method}.json", 'w') as f:
            f.write(endpoint_spec_json)
        return
    except Exception as e:
        print(f"Error serializing JSON for {method.upper()} {path}: {e}")
        return

    prompt = f"""
You are an expert in API testing and Karate DSL. Below is an OpenAPI specification for a single endpoint in JSON format. Generate a Karate feature file to test this endpoint with:

1. `Feature` title using the endpoint's summary or path/method.
2. `Background` with base URL from `servers` and JSON headers.
3. A `Scenario` for a successful request:
   - Define path parameters (e.g., `/users/{{userId}}`) with test values.
   - For POST/PUT, include a request body using schema examples or defaults.
   - Validate response status (200 for GET, 201 for POST) and required fields.
4. A `Scenario` for an error case (e.g., 400 or 404) if applicable.

OpenAPI spec:

```json
{endpoint_spec_json}
```

Return only the Karate feature file content starting with `Feature:`.
"""

    try:
        response = llm.generate(prompts=[prompt])
        feature_content = response.generations[0][0].text
        if not feature_content.strip():
            print(f"Empty response for {method.upper()} {path}")
            return
    except Exception as e:
        print(f"Error generating test for {method.upper()} {path}: {e}")
        return

    os.makedirs(output_dir, exist_ok=True)
    feature_file = f"{output_dir}/{path.replace('/', '_')}_{method}.feature"
    with open(feature_file, 'w') as f:
        f.write(feature_content)

# Generate tests for all endpoints
output_dir = 'karate_tests'
for endpoint in endpoint_specs:
    path = endpoint['path']
    method = endpoint['method']
    endpoint_spec = endpoint['spec']
    print(f"Generating test for {method.upper()} {path}")
    generate_karate_test(endpoint_spec, path, method, output_dir)
print(f"Generated tests in {output_dir}")

Generating test for POST /users
Generating test for GET /users/{userId}
Generating test for POST /orders
Generated tests in karate_tests


In [14]:
# Generate tests for all endpoints
output_dir = 'karate_tests'
endpoint_specs[0]
path = endpoint_specs[0]['path']
method = endpoint_specs[0]['method']
endpoint_spec = endpoint_specs[0]['spec']
print(f"Generating test for {method.upper()} {path}")
#generate_karate_test(endpoint_spec, path, method, output_dir)

prompt = f"""
You are an expert in API testing and Karate DSL. Below is an OpenAPI specification for a single endpoint in JSON format. Generate a Karate feature file to test this endpoint with:

1. `Feature` title using the endpoint's summary or path/method.
2. `Background` with base URL from `servers` and JSON headers.
3. A `Scenario` for a successful request:
   - Define path parameters (e.g., `/users/{{userId}}`) with test values.
   - For POST/PUT, include a request body using schema examples or defaults.
   - Validate response status (200 for GET, 201 for POST) and required fields.
4. A `Scenario` for an error case (e.g., 400 or 404) if applicable.

OpenAPI spec:

```json
{endpoint_spec}
```

Return only the Karate feature file content starting with `Feature:`.
"""

#response = llm.invoke(prompt)
response = llm.generate(prompts=[prompt])
#feature_content = response['response']
print(f"Generated tests in {response}")

Generating test for POST /users
Generated tests in generations=[[GenerationChunk(text='Here is the Karate feature file content based on the provided OpenAPI specification:\n\n```feature\nFeature: Create a new user\n\nBackground:\n* url baseDir = \'https://api.example.com/v1\'\n* header Authorization = \'Bearer YOUR_API_KEY\' // Replace YOUR_API_KEY with your actual API key\n* header Content-Type = \'application/json\'\n\nScenario: Successful creation of a new user\nGiven path \'/users\'\nAnd param userId = \'user_123\'\nAnd request {\n    "name": "John Doe",\n    "email": "john.doe@example.com",\n    "preferences": {"theme": "dark", "notifications": "enabled"},\n    "addresses": [\n        {"street": "123 Main St", "city": "Springfield", "country": "USA", "zipCode": "62701"}\n    ]\n}\nWhen method post\nThen status 201\nAnd match response.id == \'user_123\'\nAnd match response.name == \'John Doe\'\nAnd match response.email == \'john.doe@example.com\'\nAnd match response.preferences == 

In [ ]:
print(response.generations[0][0].text)

## How to Use
1. **Upload Input**: Upload `openapi.json` to Colab via file explorer or:
   ```python
   from google.colab import files
   uploaded = files.upload()
   ```
2. **Run Cells**:
   - Execute each code cell in order.
   - Check output for errors (e.g., JSON issues, Llama 3.1 failures).
3. **Check Output**: Find `.feature` files in `karate_tests` directory.
4. **Run Tests**: Download files, use Karate locally:
   ```bash
   karate karate_tests/*.feature
   ```

## Notes
- **JSON Fix**: `clean_json_string` removes invalid characters. Debug files (e.g., `debug_users_post.json`) help if errors persist.
- **Colab**: Ensure resources for Ollama. Use local machine if issues occur.
- **Output**: Expect `_users_post.feature`, `_users_{userId}_get.feature`, `_orders_post.feature`.
- **Testing**: Update API URL in feature files if needed. Use mock server if no API.
- **Debugging**: Check debug JSON files for JSON errors. Preprocess `openapi.json` if needed:
   ```python
   with open('openapi.json', 'r') as f:
       data = json.load(f)
   with open('openapi_clean.json', 'w') as f:
       json.dump(data, f, ensure_ascii=False, indent=2)
   ```